function [x,r,r_nrm,iter,flag,soltime] = mgmres(mats,b,x0,tol,maxit,restart)
% Inputs:           mats:       Array of matrices (may include a 
%                               preconditioner, at minimum contains the LHS
%                               matrix A)
%                   b:          RHS 
%                   x0:         Initial guess
%                   tol:        Convergence tolerance
%                   maxit:      Maximum number of iterations
%                   m:          Number of iterations before restarting 
%                               GMRES (set to n if using full GMRES)
%                   
% Outputs:          x:          Solution to Ax = b
%                   r:          Residual r = b - Ax
%                   r_nrm:      Vector containing residual norm for all iterations
%                   iter:       Number of iterations 
%                   flag:       1: Converged to solution; 
%                               0: Did not converge to solution within max_it
%                   soltime:    vector containing cumulative elapsed time for each
%                               iteration

tic
n = length(x0);
iter = 0;                                         
flag = 0;

r_nrm = zeros(maxit+1,1);
soltime = zeros(maxit+1,1);

x = x0;
Ax = matvec(mats,x);
r = b - Ax;
r_nrm(1) = norm( r );
soltime(1) = toc;

fprintf(1,'init. res. norm: %20.16f\n',log10(r_nrm(1)));

if ( r_nrm(1) <= tol ) 
    return
end

m = restart;
V(1:n,1:m+1) = zeros(n,m+1);
H(1:m+1,1:m) = zeros(m+1,m);
cs(1:m) = zeros(m,1);
sn(1:m) = zeros(m,1);
e1    = zeros(m+1,1);
e1(1) = 1.0;

iter = 1;
notconv = 1;
while notconv && (iter <= maxit)
    
    % Begin Arnoldi using Givens transformations
    V(:,1) = r / r_nrm(iter);
    s = r_nrm(iter)*e1; % initialize the rhs of the Hessenberg system
    
    % initialize m steps of Arnoldi
    i = 1;   
    % Initialize the Givens rotation matrix
    G = zeros(m+1,m);
    % Run m (restart) Arnoldi iterations
    while notconv && (iter <= maxit) && (i<=m)
        iter = iter+1;
        w = matvec(mats,V(:,i));
        
        for k = 1:i
            H(k,i)= V(:,k)'*w;
            G(k,i) = H(k,i);
            w = w - H(k,i)*V(:,k);
        end
        H(i+1,i) = norm( w );
        G(i+1,i) = H(i+1,i);
        if H(i+1,i) > 0
            V(:,i+1) = w / H(i+1,i);
            for k = 1:i-1 % apply Givens rotation
                temp     =  cs(k)*H(k,i) + conj(sn(k))*H(k+1,i);
                H(k+1,i) = -sn(k)*H(k,i) + conj(cs(k))*H(k+1,i);
                H(k,i)   = temp;
            end
            [cs(i),sn(i)] = rotmat( H(i,i), H(i+1,i) ); % form i-th rotation matrix
            temp   = cs(i)*s(i); % approximate residual norm
            s(i+1) = -sn(i)*s(i);
            s(i)   = temp;
            H(i,i) = cs(i)*H(i,i) + conj(sn(i))*H(i+1,i);
            H(i+1,i) = 0.0;
            r_nrm(iter)  = abs(s(i+1));
            soltime(iter) = toc;
            
            % periodically print residual information to command window
            if mod(iter,100) == 0
                fprintf(1,'GMRES out.it: %d, inn.it: %d, estimated res. norm: %20.16f\n', ...
                    iter, i, log10(r_nrm(iter)));
            end
            
            if ( r_nrm(iter) <= tol )
                notconv=0;
            else
                i = i+1;
            end 
        else
            notconv=0;
        end 
    end 
    
    if ( ~notconv )
        y = H(1:i,1:i) \ s(1:i);
        x = x + V(:,1:i)*y;
    else
        y = H(1:m,1:m) \ s(1:m);
        x = x + V(:,1:m)*y;
        i = i-1;
    end
    Ax = matvec(mats,x);
    r = b - Ax;
    
    r_nrm(iter) = norm(r); 
    soltime(iter) = toc; % Save the time elapse so far
    
    % Once m Arnoldi iterations are complete (or convergence is reached),
    % print the true residual to check against the estimated.
    fprintf(1,'GMRES out.it: %d, inn.it: %d, true res. norm: %20.16f\n', ...
        iter, i, log10(r_nrm(iter)));
    
    if (r_nrm(iter) <= tol) % check whether computed res. small enough (if s, reset converence flag)
        notconv = 0;
    else
        notconv = 1;
    end
end 

if ( r_nrm(iter) <= tol ), flag = 1; end 

r_nrm = r_nrm(1:iter); % Remove zeros from end of residual norm vector
soltime = soltime(1:iter);


function [c,s] = rotmat(a,b)
% Computes the Givens rotation matrix elements for a and b.
if ( b == 0.0 )
    c = 1.0;
    s = 0.0;
elseif ( abs(b) > abs(a) )
    temp = abs(a) / abs(b);
    c = sqrt( temp^2/(temp^2 + 1) );
    s = conj(a/(temp*b)) / sqrt(1 + temp^2);
else
    temp = abs(b) / abs(a);
    c = 1.0 / sqrt(1 + temp^2);
    s = sqrt(temp^2/(temp^2+1)) * (b/(a*temp));
end


In [86]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg

In [107]:
def gmres(A, b, x0, nmax_iter):
    
    r = b - A.dot(x0)

    x = []
    q = [0] * (nmax_iter)
    
    x.append(r)

    q[0] = r / np.linalg.norm(r)
            
    h = np.zeros((nmax_iter + 1, nmax_iter))

    for k in range(nmax_iter):
        y = A.dot(q[k])

        for j in range(k):            
            h[j, k] = np.dot(q[j].transpose(), y)
            y = y - h[j, k] * q[j]
                
        h[k + 1, k] = np.linalg.norm(y)
        if (h[k + 1, k] != 0 and k != nmax_iter - 1):
            q[k + 1] = y / h[k + 1, k]

        b = np.zeros(nmax_iter + 1)
        b[0] = np.linalg.norm(r)
        
        result = np.linalg.lstsq(h, b, rcond=-1)[0]
        
        print(np.asarray(q).transpose()[0])
                
        qres = np.dot(np.asarray(q).transpose()[0], np.asarray(result))
        
        x.append(qres.reshape(A.shape[0], 1) + x0)

    return x
        
    
A = sp.csr_matrix(np.eye(3))
b = np.random.rand(3, 1)
x_m = gmres(A, b, np.random.rand(3, 1), 2)[-1]
print('residual', b - A.dot(x_m))
print('x_m', x_m)

[[0.02469462 0.02469462]
 [0.22936395 0.22936395]
 [0.97302742 0.97302742]]
[[0.02469462 0.02469462]
 [0.22936395 0.22936395]
 [0.97302742 0.97302742]]
residual [[0.]
 [0.]
 [0.]]
x_m [[0.70682744]
 [0.10535874]
 [0.95237093]]
